Added Semantic Chunking, and Reranking

## Naive RAG : QA Pipeline 

In [1]:
#Init Libraries

#Env Variables
from dotenv import load_dotenv
import os

#Embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

#LlamaIndex RAG Core Libraries
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings

#LLM
from groq import Groq
from groqllm import GroqLLM

#VectorStorage
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate,Document 
import chromadb

#Chunking
from llama_index.core.node_parser import  SemanticSplitterNodeParser

#DML
import pandas as pd

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Loading RAG Components

In [2]:
#Loading Embedding Model
embed_model = HuggingFaceEmbedding(model_name='Snowflake/snowflake-arctic-embed-s' 
                                   ,trust_remote_code=True
                                   )

#Loading LLM from Groq
llm = GroqLLM(model_name = "llama3-8b-8192"
             ,client =Groq(api_key=os.getenv("GROQ_API_KEY"))
             ,temperature =0.1)

In [3]:
#Setting up temporary ChromaDB
db = chromadb.EphemeralClient() #Makes a temporary client which is not on disk
chroma_collection = db.get_or_create_collection("temp")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

### Setting LlamaIndex Contexts

In [4]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
Settings.llm = llm
Settings.embed_model = embed_model

#### Loading Context

In [5]:
df = pd.read_csv('../data/MainDataset/data/Context.csv')

context_list = []
for contexts in df['context']:
    context_list.append(Document(text=contexts))

In [6]:
# Chunking Pipeline
pipeline = IngestionPipeline(
    transformations=[
      SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)
    ],
    vector_store=vector_store,
)

documents = pipeline.run(documents=context_list)

In [7]:
df_test = pd.Series([node.text for node in documents])

In [8]:
df_test.str.len().mean()

1030.0277777777778

In [9]:
reranker_model = FlagEmbeddingReranker(model="mixedbread-ai/mxbai-embed-large-v1", top_n=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mixedbread-ai/mxbai-embed-large-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#Create VectorStoreIndex for RAG
index = VectorStoreIndex(documents, storage_context=storage_context,similarity_top_k=10,node_postprocessors=[reranker_model]) 

In [11]:
#Turn Index into a Query Engine
query_engine = index.as_query_engine(streaming=True,similarity_top_k=5)

In [12]:
#Set a RAG Prompt for the System
qa_prompt_template_str ="""
Context: {context_str}
Instructions:
- You are under an examination setting, your goal is to accurately answer the questions to your best ability.
- You will be evaluated after the exam.
- Utilize the context provided for accurate and specific information.
- Keep your answers short.
- You will be penalised if you are writing unnessecary words
Question: {query_str}
"""

In [13]:
#Update initial prompt and remove the refining prompt
qa_prompt_template = PromptTemplate(qa_prompt_template_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":qa_prompt_template}
)


query_engine.update_prompts(
    {"response_synthesizer:refine_template":PromptTemplate("")}
)


### Loading Questions

In [14]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [15]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)
    

In [16]:
import time

for i in range(len(question_list)): 
    time.sleep(3)
    print(question_list[i])
    response = query_engine.query(question_list[i])
    response.print_response_stream()
    rowIndex = df_qa.index[i]

    df_qa.at[rowIndex, 'gen_answer'] = response.response_txt
    print('')
    print("--"*100)
    print(f'{i}/{128}')

who introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands? brainly
Joseph Lister introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands, following the publication of his paper "Antiseptic Principle of the Practice of Surgery" in 1867.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
president adams
John Quincy Adams was the sixth President of the United States, serving from 1825 to 1829.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1/128
define semitic
According to the context, a Semitic person is defined as:

"1a: a member of any of a number of peoples of ancient southwestern Asia including the Akkadian

In [17]:
df_qa.head()

,question,answers,gen_answer
0,"who introduced modern methods of surgery, such...",Joseph Lister,Joseph Lister introduced modern methods of sur...
1,president adams,John Quincy Adams was an American statesman wh...,John Quincy Adams was the sixth President of t...
2,define semitic,A member of any of a number of peoples of anci...,"According to the context, a Semitic person is ..."
3,what is wudfhost service,It is a set of Microsoft tools that aid in the...,The WUDFHost service is a Windows Driver Found...
4,what is an overactive pancreas,An overactive pancrease would produce more enz...,An overactive pancreas would produce more enzy...


In [18]:
df_qa.to_csv(f'../data/MainDataset/results/Official/Advanced_RAG_3.csv',index=False)

: 